In [12]:
import torch

import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from src.training.datasets import CustomImageFolder
import os
from settings import settings
import torchvision.models as models

In [13]:
images_path=os.path.join(settings.TRAINING.data_root, "training/ternary")
random_flip=True
target_flips={1: 2, 2: 1}

In [14]:
dataset = CustomImageFolder(
            root=images_path,
            target_flips=target_flips,
            transform=transforms.Compose([
                transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
                transforms.Resize((224, 224)),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ],

            )
        )

In [15]:
batch_size = 16
test_pct = 20
test_size = int(len(dataset)*test_pct/100.0)

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - test_size, test_size])

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=0,
)

In [16]:
# number of classes
K = 3 #len(set(train_dataset.targets.numpy()))
print("number of classes:", K)

number of classes: 3


In [19]:
dataset.__getitem__(0)[0].shape

torch.Size([3, 224, 224])

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [7]:
# The same model! Using the newly introduced "Flatten"
class CNN(nn.Module):
  def __init__(self, K):
    super(CNN, self).__init__()
    layers = [
        (3,32,6,2,2),
        (32,64,6,2,1),
        (64,32,1,1,0)
    ]
    size = 224
    out = 0
    self.conv_layers = nn.Sequential()
    for i,l in enumerate(layers):
        size = int((size + 2*l[4] - l[2])/l[3] + 1)
        self.conv_layers.add_module(
            module=nn.Conv2d(in_channels=l[0], out_channels=l[1], kernel_size=l[2], stride=l[3], padding=l[4]),
            name=f'c{i}'
        )
        self.conv_layers.add_module(module=nn.SiLU(inplace=True), name=f's{i}')
        out = l[1]

    print(size)
    print(out)

    # http://deeplearning.net/software/theano/tutorial/conv_arithmetic.html
    # "No zero padding, non-unit strides"
    # https://pytorch.org/docs/stable/nn.html
    self.dense_layers = nn.Sequential(
      nn.Dropout(0.2),
      nn.Linear(size*size*out, 128),
      nn.SiLU(),
      nn.Linear(128, 3),
    )

  def forward(self, X):
    out = self.conv_layers(X)
    # print(out.shape)
    out = out.view(out.size(0), -1)
    # print(out.shape)
    out = self.dense_layers(out)
    return out

In [8]:
#model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
#model.eval()

In [9]:
model = CNN(K)
#model = torchvision.models.resnet18(pretrained=True)
#model.fc = nn.Linear(512,3)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)

criterion = nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

55
32
cpu


In [10]:
def batch_gd(model, criterion, optimizer, train_loader, test_loader, epochs):
  train_losses = np.zeros(epochs)
  test_losses = np.zeros(epochs)

  for it in range(epochs):
    model.train()
    t0 = datetime.now()
    train_loss = []
    for inputs, targets in train_loader:
      # move data to GPU
      inputs, targets = inputs.to(device), targets.to(device)

      # zero the parameter gradients
      optimizer.zero_grad()

      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, targets)

      # Backward and optimize
      loss.backward()
      optimizer.step()

      train_loss.append(loss.item())

    # Get train loss and test loss
    train_loss = np.mean(train_loss) # a little misleading

    model.eval()
    test_loss = []
    for inputs, targets in test_loader:
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = model(inputs)
      loss = criterion(outputs, targets)
      test_loss.append(loss.item())
    test_loss = np.mean(test_loss)

    # Save losses
    train_losses[it] = train_loss
    test_losses[it] = test_loss

    dt = datetime.now() - t0
    print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
      Test Loss: {test_loss:.4f}, Duration: {dt}')

  return train_losses, test_losses

In [11]:
train_losses, test_losses = batch_gd(
    model, criterion, optimizer, train_loader, test_loader, epochs=30)

KeyboardInterrupt: 

In [ ]:
# Plot the train loss and test loss per iteration
plt.plot(train_losses, label='train loss')
plt.plot(test_losses, label='test loss')
plt.legend()
plt.show()